# Welcome to the OpenCite OpenRouter Bulk Runner!

Track LLM citations across multiple models with OpenRouter's unified API.

## What you can do

- **Multi-model citation tracking** - Compare how different models cite sources
- **Batch processing** - Run multiple prompts with personas and multi-turn conversations
- **Citation analysis** - Domain, page, and prompt-level reports
- **Location bias** - Test how location affects citations

## How it works

| Step | Cell | Description |
|------|------|-------------|
| 1 | Workspace Setup | Choose where to store your project |
| 2 | Download Scripts | Get the latest scripts from GitHub |
| 3 | Connect to OpenRouter | Enter API key and select model |
| 4 | Upload Prompts | Upload your CSV with prompts and personas |
| 5 | Run Batch | Execute prompts and collect citations |
| 6 | Browse Results | View and filter your results |
| 7 | Analyse Citations | Generate domain, page, or prompt reports |

## Supported Models

Any model on OpenRouter with the `:online` suffix:
- `openai/gpt-4o:online`
- `anthropic/claude-sonnet-4:online`
- `perplexity/sonar-pro:online`
- And many more...

## Links

- [OpenCite GitHub](https://github.com/smartaces/opencite)
- [OpenRouter Docs](https://openrouter.ai/docs)

---

**Run each cell in order to get started.**

In [ ]:
# =============================================================================
# CELL 2: WORKSPACE SETUP
# =============================================================================
# This cell does two things:
# 1. Lets you choose where to store your project (Google Drive recommended)
# 2. Creates all the necessary subfolders including the scripts folder
#
# Run this cell first, then click "Select Base Location" and follow the prompts.
# =============================================================================

import json
import os
import sys
from datetime import datetime
from pathlib import Path

import ipywidgets as widgets
from IPython.display import clear_output, display

try:
    from google.colab import drive  # type: ignore
except ImportError:
    drive = None

IN_COLAB = "google.colab" in sys.modules


def _default_project_name() -> str:
    return f"project_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"


def _ensure_drive_mounted() -> Path:
    if drive is None:
        raise RuntimeError("google.colab.drive is unavailable in this environment.")
    mount_point = Path("/content/drive")
    if not mount_point.exists() or not os.path.ismount(mount_point):
        print("Mounting Google Drive...")
        drive.mount(str(mount_point))
    return mount_point / "MyDrive"


def _scan_existing_projects(workspace_root: Path) -> list:
    """Scan workspace_root for existing project folders, sorted by most recently modified."""
    if not workspace_root.exists():
        return []

    folders = []
    try:
        for item in workspace_root.iterdir():
            if item.is_dir() and not item.name.startswith('.'):
                try:
                    mtime = item.stat().st_mtime
                    folders.append((item, mtime))
                except OSError:
                    continue
    except PermissionError:
        return []

    folders.sort(key=lambda x: x[1], reverse=True)

    result = []
    for folder_path, mtime in folders:
        modified_date = datetime.fromtimestamp(mtime).strftime('%Y-%m-%d %H:%M')
        label = f"{folder_path.name}  (modified: {modified_date})"
        result.append((label, str(folder_path)))

    return result


def _build_location_options():
    if IN_COLAB:
        return [
            ("Google Drive Folder (/content/drive/MyDrive)", "drive"),
            ("Google Colab Temporary Folder (/content)", "colab"),
            ("Local Folder (current directory)", "local"),
        ]
    else:
        return [
            ("Local Folder (current directory)", "local"),
        ]


def _create_workspace_structure(workspace_path: Path) -> dict:
    """Create all workspace subfolders and return PATHS dict."""
    subfolders = {
        "scripts": workspace_path / "scripts",
        "search_results": workspace_path / "search_results",
        "extracted_raw": workspace_path / "extracted_raw",
        "csv_output": workspace_path / "csv_output",
        "grabbed": workspace_path / "grabbed",
        "terms_lists": workspace_path / "terms_lists",
        "logs": workspace_path / "logs",
    }

    for path in subfolders.values():
        path.mkdir(parents=True, exist_ok=True)

    return {name: str(path) for name, path in subfolders.items()}


def _write_workspace_config(workspace_path: Path, paths: dict) -> Path:
    """Write workspace config file and return its path."""
    config = {
        "workspace_root": str(workspace_path),
        "paths": paths,
    }
    config_path = workspace_path / "workspace_config.json"
    with open(config_path, "w", encoding="utf-8") as fp:
        json.dump(config, fp, indent=2)
    return config_path


def _setup_smart_file_handling(workspace_path: Path):
    """Override builtins.open to resolve relative paths to workspace."""
    import builtins
    _original_open = builtins.open

    def smart_open(file, *args, **kwargs):
        file_path = Path(file)
        if not file_path.is_absolute():
            file_path = workspace_path / file_path
        file_path.parent.mkdir(parents=True, exist_ok=True)
        return _original_open(file_path, *args, **kwargs)

    builtins.open = smart_open


def configure_workspace(default_folder: str = "opencite_openrouter_workspace") -> None:
    state = {"base_path": None}

    header = widgets.HTML("<h3>Workspace Setup</h3>")

    location_dropdown = widgets.Dropdown(
        options=_build_location_options(),
        value=_build_location_options()[0][1],
        description="Location:",
        style={'description_width': '80px'},
        layout=widgets.Layout(width="450px"),
    )
    select_base_button = widgets.Button(
        description="Select Location",
        icon="map",
        button_style="info",
        layout=widgets.Layout(width="140px"),
    )
    base_status_output = widgets.Output()

    stage_container = widgets.VBox(
        [
            header,
            widgets.HTML("<p>Choose where to store your project. Google Drive is recommended for persistence.</p>"),
            widgets.HBox([location_dropdown, select_base_button], layout=widgets.Layout(gap='8px')),
            base_status_output,
        ],
        layout=widgets.Layout(width="100%", gap="8px"),
    )

    def _render_folder_stage(base_path: Path):
        workspace_root = (base_path / default_folder).expanduser().resolve()
        workspace_root.mkdir(parents=True, exist_ok=True)

        existing_projects = _scan_existing_projects(workspace_root)
        has_existing = len(existing_projects) > 0

        mode_state = {"mode": "existing" if has_existing else "new"}

        existing_dropdown = widgets.Dropdown(
            options=[("-- Select a project --", "")] + existing_projects,
            value="",
            style={'description_width': '100px'},
            layout=widgets.Layout(width="500px"),
        )
        switch_to_new_button = widgets.Button(
            description="Create new project instead",
            button_style="",
            icon="plus",
            layout=widgets.Layout(width="220px"),
        )
        existing_section = widgets.VBox([
            widgets.HTML(f"<b>Continue an existing project</b> ({len(existing_projects)} found)"),
            existing_dropdown,
            switch_to_new_button,
        ])

        project_name_input = widgets.Text(
            value=_default_project_name(),
            placeholder="project_YYYYMMDD_HHMMSS",
            style={'description_width': '100px'},
            layout=widgets.Layout(width="450px"),
        )
        switch_to_existing_button = widgets.Button(
            description="Select existing project instead",
            button_style="",
            icon="folder-open",
            layout=widgets.Layout(width="240px"),
        )
        new_section = widgets.VBox([
            widgets.HTML("<b>Create a new project</b>"),
            project_name_input,
            switch_to_existing_button if has_existing else widgets.HTML(""),
        ])

        base_path_label = widgets.HTML(f"<b>Workspace location:</b> {workspace_root}")

        action_button = widgets.Button(
            description="Open Project" if has_existing else "Create Project",
            button_style="primary",
            icon="folder-open" if has_existing else "plus",
            layout=widgets.Layout(width="150px"),
        )
        status_output = widgets.Output()

        mode_container = widgets.VBox([existing_section if has_existing else new_section])

        def _switch_to_new(_):
            mode_state["mode"] = "new"
            mode_container.children = [new_section]
            action_button.description = "Create Project"
            action_button.icon = "plus"
            existing_dropdown.value = ""

        def _switch_to_existing(_):
            mode_state["mode"] = "existing"
            mode_container.children = [existing_section]
            action_button.description = "Open Project"
            action_button.icon = "folder-open"

        def _handle_action(_):
            with status_output:
                clear_output()
                try:
                    if mode_state["mode"] == "existing":
                        selected = existing_dropdown.value
                        if not selected:
                            raise RuntimeError("Select a project from the dropdown.")
                        workspace_path = Path(selected).expanduser().resolve()
                    else:
                        project_name = project_name_input.value.strip()
                        if not project_name:
                            raise RuntimeError("Enter a project folder name.")
                        workspace_path = (workspace_root / project_name).expanduser().resolve()

                    paths = _create_workspace_structure(workspace_path)
                    config_path = _write_workspace_config(workspace_path, paths)

                    os.environ["WORKSPACE_ROOT"] = str(workspace_path)
                    os.environ["WORKSPACE_CONFIG"] = str(config_path)

                    import __main__
                    __main__.WORKSPACE_ROOT = workspace_path
                    __main__.PATHS = {k: Path(v) for k, v in paths.items()}
                    __main__.WORKSPACE_CONFIG = config_path

                    globals()["WORKSPACE_ROOT"] = workspace_path
                    globals()["PATHS"] = {k: Path(v) for k, v in paths.items()}
                    globals()["WORKSPACE_CONFIG"] = config_path

                    _setup_smart_file_handling(workspace_path)

                    print(f"Workspace ready: {workspace_path}")
                    print(f"Folders created:")
                    for name in paths:
                        print(f"   - {name}")
                    print(f"\nRun the next cell to download/update scripts.")

                except Exception as exc:
                    print(f"Error: {exc}")

        switch_to_new_button.on_click(_switch_to_new)
        switch_to_existing_button.on_click(_switch_to_existing)
        action_button.on_click(_handle_action)

        stage_container.children = [
            header,
            widgets.HTML("<p><strong>Location selected.</strong> Now choose or create a project.</p>"),
            base_path_label,
            widgets.HTML("<hr style='margin: 12px 0;'>"),
            mode_container,
            widgets.HTML("<hr style='margin: 12px 0;'>"),
            action_button,
            status_output,
        ]

    def _handle_base_select(_):
        with base_status_output:
            clear_output()
            try:
                selection = location_dropdown.value
                if selection == "colab":
                    base_path = Path("/content")
                elif selection == "drive":
                    base_path = _ensure_drive_mounted()
                elif selection == "local":
                    base_path = Path.cwd()
                else:
                    base_path = Path.cwd()

                base_path = base_path.expanduser().resolve()
                if not base_path.exists():
                    base_path.mkdir(parents=True, exist_ok=True)
                state["base_path"] = base_path
                print(f"Location ready: {base_path}")
                _render_folder_stage(base_path)
            except Exception as exc:
                state["base_path"] = None
                print(f"Error: {exc}")

    select_base_button.on_click(_handle_base_select)

    display(stage_container)


configure_workspace()

In [ ]:
# =============================================================================
# CELL 3: DOWNLOAD SCRIPTS FROM GITHUB
# =============================================================================
# Downloads all cell_*.py scripts from the GitHub repository to your
# workspace/scripts/ folder. Run this after setting up your workspace.
# =============================================================================

import hashlib
import json
import os
from pathlib import Path
from urllib.request import urlopen, Request
from urllib.error import URLError

import ipywidgets as widgets
from IPython.display import display, clear_output

# GitHub repository configuration
GITHUB_REPO = "smartaces/opencite"
GITHUB_BRANCH = "main"
GITHUB_FOLDER = "openrouter_bulk_loader/github_scripts"

# Scripts to download
SCRIPT_FILES = [
    "cell_00_openrouter_pip_installs.py",
    "cell_02_openrouter_api_key.py",
    "cell_03_openrouter_report_helper.py",
    "cell_04_openrouter_search_agent.py",
    "cell_05_openrouter_csv_loader.py",
    "cell_06_openrouter_batch_runner.py",
    "cell_07_openrouter_results_viewer.py",
    "cell_08_openrouter_dataset_builder.py",
    "cell_09_openrouter_domain_report.py",
    "cell_10_openrouter_page_report.py",
    "cell_11_openrouter_prompt_report.py",
]


def _get_scripts_path() -> Path:
    """Get the scripts folder path from workspace config."""
    if 'PATHS' in globals():
        return Path(PATHS['scripts'])

    config_path = Path(os.environ.get('WORKSPACE_CONFIG', ''))
    if not config_path.is_file():
        raise RuntimeError("Workspace not configured. Run the workspace setup cell first.")

    with open(config_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return Path(data['paths']['scripts'])


def _get_github_raw_url(filename: str) -> str:
    """Build GitHub raw content URL for a file."""
    return f"https://raw.githubusercontent.com/{GITHUB_REPO}/{GITHUB_BRANCH}/{GITHUB_FOLDER}/{filename}"


def _download_file(url: str, dest_path: Path) -> bool:
    """Download a file from URL to destination path."""
    try:
        req = Request(url, headers={'User-Agent': 'OpenCite-Bulk-Loader/1.0'})
        with urlopen(req, timeout=30) as response:
            content = response.read()
        dest_path.write_bytes(content)
        return True
    except URLError as e:
        print(f"Failed to download {url}: {e}")
        return False
    except Exception as e:
        print(f"Error saving {dest_path.name}: {e}")
        return False


def _get_file_hash(path: Path) -> str:
    """Get MD5 hash of a file."""
    if not path.exists():
        return ""
    return hashlib.md5(path.read_bytes()).hexdigest()


def _download_all_scripts(scripts_path: Path, force: bool = False) -> dict:
    """Download all scripts, optionally forcing re-download."""
    results = {"downloaded": [], "skipped": [], "failed": []}

    scripts_path.mkdir(parents=True, exist_ok=True)

    for filename in SCRIPT_FILES:
        dest_path = scripts_path / filename
        url = _get_github_raw_url(filename)

        if dest_path.exists() and not force:
            try:
                req = Request(url, headers={'User-Agent': 'OpenCite-Bulk-Loader/1.0'})
                with urlopen(req, timeout=30) as response:
                    remote_content = response.read()
                remote_hash = hashlib.md5(remote_content).hexdigest()
                local_hash = _get_file_hash(dest_path)

                if remote_hash == local_hash:
                    results["skipped"].append(filename)
                    continue
                else:
                    dest_path.write_bytes(remote_content)
                    results["downloaded"].append(filename)
            except Exception:
                results["skipped"].append(filename)
                continue
        else:
            if _download_file(url, dest_path):
                results["downloaded"].append(filename)
            else:
                results["failed"].append(filename)

    return results


scripts_path = _get_scripts_path()
status_output = widgets.Output()

check_button = widgets.Button(
    description="Check for Updates",
    button_style="info",
    icon="refresh",
    layout=widgets.Layout(width="180px"),
)

force_button = widgets.Button(
    description="Force Re-download All",
    button_style="warning",
    icon="download",
    layout=widgets.Layout(width="180px"),
)


def _handle_check(_):
    with status_output:
        clear_output()
        print("Checking for updates...")
        results = _download_all_scripts(scripts_path, force=False)
        print(f"\nResults:")
        if results["downloaded"]:
            print(f"  Updated: {len(results['downloaded'])} files")
            for f in results["downloaded"]:
                print(f"    - {f}")
        if results["skipped"]:
            print(f"  Up to date: {len(results['skipped'])} files")
        if results["failed"]:
            print(f"  Failed: {len(results['failed'])} files")
            for f in results["failed"]:
                print(f"    - {f}")
        print(f"\nScripts location: {scripts_path}")


def _handle_force(_):
    with status_output:
        clear_output()
        print("Force downloading all scripts...")
        results = _download_all_scripts(scripts_path, force=True)
        print(f"\nResults:")
        print(f"  Downloaded: {len(results['downloaded'])} files")
        if results["failed"]:
            print(f"  Failed: {len(results['failed'])} files")
            for f in results["failed"]:
                print(f"    - {f}")
        print(f"\nScripts location: {scripts_path}")


check_button.on_click(_handle_check)
force_button.on_click(_handle_force)

controls = widgets.VBox([
    widgets.HTML("<h3>Download Scripts</h3>"),
    widgets.HTML("<p>Download or update the OpenCite scripts from GitHub.</p>"),
    widgets.HTML(f"<p><b>Scripts folder:</b> {scripts_path}</p>"),
    widgets.HBox([check_button, force_button], layout=widgets.Layout(gap='8px')),
    status_output,
])

display(controls)

print("Downloading scripts...")
results = _download_all_scripts(scripts_path, force=False)
if results["downloaded"]:
    print(f"Downloaded {len(results['downloaded'])} scripts")
if results["skipped"]:
    print(f"Already up to date: {len(results['skipped'])} scripts")
if results["failed"]:
    print(f"Failed to download: {results['failed']}")
print(f"\nScripts ready in: {scripts_path}")
print("Run the next cell to connect to OpenRouter.")

In [ ]:
# =============================================================================
# CELL 4: CONNECT TO OPENROUTER
# =============================================================================
# This cell loads the core scripts and connects to the OpenRouter API.
# It will display a model selector UI when complete.
# =============================================================================

import json
import os
import subprocess
import sys
from pathlib import Path

def _get_scripts_path() -> Path:
    """Get the scripts folder path from workspace config."""
    if 'PATHS' in globals():
        return Path(PATHS['scripts'])

    config_path = Path(os.environ.get('WORKSPACE_CONFIG', ''))
    if not config_path.is_file():
        raise RuntimeError("Workspace not configured. Run the workspace setup cell first.")

    with open(config_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return Path(data['paths']['scripts'])


scripts_path = _get_scripts_path()

# 1. Install dependencies (silent)
print("Installing dependencies...")
pip_script = scripts_path / "cell_00_openrouter_pip_installs.py"
if pip_script.exists():
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "--quiet",
         "openai", "rich", "python-dotenv", "ipywidgets", "plotly", "posthog"],
        capture_output=True
    )
    print("Dependencies installed.")
else:
    print("Warning: pip install script not found, attempting to continue...")

# 2. Load API key configuration
print("Loading API key...")
api_key_script = scripts_path / "cell_02_openrouter_api_key.py"
if api_key_script.exists():
    exec(compile(open(api_key_script, encoding='utf-8').read(), api_key_script, 'exec'))
else:
    raise RuntimeError(f"API key script not found: {api_key_script}")

# 3. Load ReportHelper class
print("Loading ReportHelper...")
report_helper_script = scripts_path / "cell_03_openrouter_report_helper.py"
if report_helper_script.exists():
    exec(compile(open(report_helper_script, encoding='utf-8').read(), report_helper_script, 'exec'))
else:
    raise RuntimeError(f"Report helper script not found: {report_helper_script}")

# 4. Load Dataset Builder (for filter helpers used by reports)
print("Loading Dataset Builder...")
dataset_builder_script = scripts_path / "cell_08_openrouter_dataset_builder.py"
if dataset_builder_script.exists():
    exec(compile(open(dataset_builder_script, encoding='utf-8').read(), dataset_builder_script, 'exec'))
else:
    print("Warning: Dataset builder script not found, reports may have limited functionality.")

# 5. Load Search Agent (displays model selector UI)
print("Loading Search Agent...")
search_agent_script = scripts_path / "cell_04_openrouter_search_agent.py"
if search_agent_script.exists():
    exec(compile(open(search_agent_script, encoding='utf-8').read(), search_agent_script, 'exec'))
else:
    raise RuntimeError(f"Search agent script not found: {search_agent_script}")

print("\nOpenRouter connection ready!")
print("Run the next cell to upload your prompts CSV.")

In [ ]:
# =============================================================================
# CELL 5: UPLOAD PROMPTS
# =============================================================================
# Upload your CSV file with prompts, personas, and run/turn configurations.
# =============================================================================

import json
import os
from pathlib import Path

def _get_scripts_path() -> Path:
    """Get the scripts folder path from workspace config."""
    if 'PATHS' in globals():
        return Path(PATHS['scripts'])

    config_path = Path(os.environ.get('WORKSPACE_CONFIG', ''))
    if not config_path.is_file():
        raise RuntimeError("Workspace not configured. Run the workspace setup cell first.")

    with open(config_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return Path(data['paths']['scripts'])


scripts_path = _get_scripts_path()

csv_loader_script = scripts_path / "cell_05_openrouter_csv_loader.py"
if csv_loader_script.exists():
    exec(compile(open(csv_loader_script, encoding='utf-8').read(), csv_loader_script, 'exec'))
else:
    raise RuntimeError(f"CSV loader script not found: {csv_loader_script}")

In [ ]:
# =============================================================================
# CELL 6: RUN BATCH
# =============================================================================
# Execute your prompts through the OpenRouter search agent.
# =============================================================================

import json
import os
from pathlib import Path

def _get_scripts_path() -> Path:
    """Get the scripts folder path from workspace config."""
    if 'PATHS' in globals():
        return Path(PATHS['scripts'])

    config_path = Path(os.environ.get('WORKSPACE_CONFIG', ''))
    if not config_path.is_file():
        raise RuntimeError("Workspace not configured. Run the workspace setup cell first.")

    with open(config_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return Path(data['paths']['scripts'])


scripts_path = _get_scripts_path()

batch_runner_script = scripts_path / "cell_06_openrouter_batch_runner.py"
if batch_runner_script.exists():
    exec(compile(open(batch_runner_script, encoding='utf-8').read(), batch_runner_script, 'exec'))
else:
    raise RuntimeError(f"Batch runner script not found: {batch_runner_script}")

In [ ]:
# =============================================================================
# CELL 7: BROWSE RESULTS
# =============================================================================
# Browse and filter your batch run results.
# =============================================================================

import json
import os
from pathlib import Path

def _get_scripts_path() -> Path:
    """Get the scripts folder path from workspace config."""
    if 'PATHS' in globals():
        return Path(PATHS['scripts'])

    config_path = Path(os.environ.get('WORKSPACE_CONFIG', ''))
    if not config_path.is_file():
        raise RuntimeError("Workspace not configured. Run the workspace setup cell first.")

    with open(config_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return Path(data['paths']['scripts'])


scripts_path = _get_scripts_path()

results_viewer_script = scripts_path / "cell_07_openrouter_results_viewer.py"
if results_viewer_script.exists():
    exec(compile(open(results_viewer_script, encoding='utf-8').read(), results_viewer_script, 'exec'))
else:
    raise RuntimeError(f"Results viewer script not found: {results_viewer_script}")

In [ ]:
# =============================================================================
# CELL 8: ANALYSE CITATIONS (REPORTS)
# =============================================================================
# Generate domain, page, or prompt-level reports from your batch results.
# =============================================================================

import json
import os
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output


def _get_scripts_path() -> Path:
    """Get the scripts folder path from workspace config."""
    if 'PATHS' in globals():
        return Path(PATHS['scripts'])

    config_path = Path(os.environ.get('WORKSPACE_CONFIG', ''))
    if not config_path.is_file():
        raise RuntimeError("Workspace not configured. Run the workspace setup cell first.")

    with open(config_path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return Path(data['paths']['scripts'])


scripts_path = _get_scripts_path()

REPORT_OPTIONS = [
    ("Domain Report - Aggregate citations by domain", "domain"),
    ("Page Report - Aggregate citations by URL", "page"),
    ("Prompt Report - Analyse citations by prompt", "prompt"),
]

report_dropdown = widgets.Dropdown(
    options=REPORT_OPTIONS,
    value="domain",
    description="Report:",
    style={'description_width': '60px'},
    layout=widgets.Layout(width='450px'),
)

load_button = widgets.Button(
    description="Load Report",
    button_style="primary",
    icon="chart-bar",
    layout=widgets.Layout(width="140px"),
)

report_output = widgets.Output()

_loaded_reports = set()


def _handle_load(_):
    report_type = report_dropdown.value

    with report_output:
        clear_output()
        print(f"Loading {report_type} report...")

        try:
            if report_type == "domain":
                script_path = scripts_path / "cell_09_openrouter_domain_report.py"
            elif report_type == "page":
                script_path = scripts_path / "cell_10_openrouter_page_report.py"
            elif report_type == "prompt":
                script_path = scripts_path / "cell_11_openrouter_prompt_report.py"
            else:
                raise ValueError(f"Unknown report type: {report_type}")

            if not script_path.exists():
                raise RuntimeError(f"Report script not found: {script_path}")

            exec(compile(open(script_path, encoding='utf-8').read(), script_path, 'exec'), globals())
            _loaded_reports.add(report_type)

        except Exception as exc:
            print(f"Error loading report: {exc}")


load_button.on_click(_handle_load)

controls = widgets.VBox([
    widgets.HTML("<h3>Analyse Citations</h3>"),
    widgets.HTML("<p>Select a report type and click Load to generate the analysis.</p>"),
    widgets.HBox([report_dropdown, load_button], layout=widgets.Layout(gap='8px', align_items='center')),
    widgets.HTML("<hr style='margin: 12px 0;'>"),
    report_output,
])

display(controls)